# Load Package

In [1]:
from lxml.html import fromstring

import pandas as pd
import numpy as np

import requests
import time
import os

# 데이터 불러오기

In [2]:
hitter = pd.read_csv('선수데이터(전처리완료)/타자.csv')[['선수명', '생년월일']].drop_duplicates()
pitcher = pd.read_csv('선수데이터(전처리완료)/투수.csv')[['선수명', '생년월일']].drop_duplicates()
df = pd.concat([hitter, pitcher]).reset_index(drop=True)
df

,선수명,생년월일
0,김경언,1982-12-07
1,김민철,1982-03-14
2,김상훈,1977-10-27
3,김인철,1971-07-14
4,김종국,1973-09-14
...,...,...
1583,김기탁,1998-09-30
1584,오동욱,2001-02-04
1585,윤대경,1994-04-09
1586,장웅정,1997-04-11


In [3]:
PLAYER_NAMES = list(df['선수명'])
BIRTHDAYS = list(df['생년월일'])

# Crawling 
 - STATIZ에서 데뷔년도 찾아옥;
 - 생년월일의 오류가 간간히 존재하여 직접 수정

In [4]:
cnt = 0
t1 = time.time()

for PLAYER_NAME, BIRTHDAY in zip(PLAYER_NAMES, BIRTHDAYS):

    # 선수 count용
    cnt += 1
    tmp = pd.DataFrame()
    
    try:
        # 1차 url을 대입하여 수집
        url = f'http://www.statiz.co.kr/player.php?name={PLAYER_NAME}&search='
        res = requests.get(url)
        parser = fromstring(res.text)

        crawl_birthday = parser.xpath('/html/body/div[1]/div[1]/div/section[2]/div/div[1]/div/div[1]/div/div[2]/ul/li[1]')[0].text_content()[5:].replace('년 ','-').replace('월 ','-').replace('일','')
        crawl_startyear = parser.xpath(f'/html/body/div[1]/div[1]/div/section[2]/div/div[1]/div/div[1]/div/div[2]/ul/li[6]')[0].text_content()[5:].split('년')[0]

        tmp = pd.DataFrame({'선수명':[PLAYER_NAME],
                            '생년월일(KBO)':[BIRTHDAY],
                           '생년월일(STATIZ)':[crawl_birthday],
                           '데뷔년도':[crawl_startyear]})

        if cnt == 1:
            final = tmp.copy()
        else:
            final = pd.concat([final, tmp]).reset_index(drop=True)
            
    except:
        try: # 2차 url을 대입하여 수집
            url = f'http://www.statiz.co.kr/player.php?name={PLAYER_NAME}&birth={BIRTHDAY}'
            res = requests.get(url)
            parser = fromstring(res.text)

            crawl_birthday = parser.xpath('/html/body/div[1]/div[1]/div/section[2]/div/div[1]/div/div[1]/div/div[2]/ul/li[1]')[0].text_content()[5:].replace('년 ','-').replace('월 ','-').replace('일','')
            crawl_startyear = parser.xpath(f'/html/body/div[1]/div[1]/div/section[2]/div/div[1]/div/div[1]/div/div[2]/ul/li[6]')[0].text_content()[5:].split('년')[0]

            tmp = pd.DataFrame({'선수명':[PLAYER_NAME],
                                '생년월일(KBO)':[BIRTHDAY],
                               '생년월일(STATIZ)':[crawl_birthday],
                               '데뷔년도':[crawl_startyear]})

            if cnt == 1:
                final = tmp.copy()
            else:
                final = pd.concat([final, tmp]).reset_index(drop=True)
                
        except: # 정말 정보가 없음
            print(f'== {PLAYER_NAME}({BIRTHDAY}) 수집불가 ==')
    
    
    if cnt % 100 == 0:
        print(f'{cnt}명 완료 == 경과시간: {round(time.time()-t1)}초 == 10초 휴식')
        time.sleep(10)

100명 완료 == 경과시간: 746초 == 10초 휴식
200명 완료 == 경과시간: 1406초 == 10초 휴식
300명 완료 == 경과시간: 1921초 == 10초 휴식
== 유재원(1983-12-13) 수집불가 ==
== 박영복(1983-04-12) 수집불가 ==
400명 완료 == 경과시간: 2460초 == 10초 휴식
== 한동민(1989-08-09) 수집불가 ==
500명 완료 == 경과시간: 2821초 == 10초 휴식
== 김경근(1988-09-26) 수집불가 ==
600명 완료 == 경과시간: 3052초 == 10초 휴식
== 나종덕(1998-03-16) 수집불가 ==
700명 완료 == 경과시간: 3274초 == 10초 휴식
== 윤대영(1994-01-31) 수집불가 ==
800명 완료 == 경과시간: 3365초 == 10초 휴식
== 김현민(2000-02-23) 수집불가 ==
== 이용훈(1977-07-14) 수집불가 ==
900명 완료 == 경과시간: 3684초 == 10초 휴식
== 장기영(1982-05-09) 수집불가 ==
1000명 완료 == 경과시간: 3951초 == 10초 휴식
1100명 완료 == 경과시간: 4240초 == 10초 휴식
== 전태현(1989-06-20) 수집불가 ==
== 오병일(1990-04-20) 수집불가 ==
1200명 완료 == 경과시간: 4486초 == 10초 휴식
== 박동욱(1985-04-06) 수집불가 ==
== 정진(1979-08-06) 수집불가 ==
1300명 완료 == 경과시간: 4674초 == 10초 휴식
1400명 완료 == 경과시간: 4840초 == 10초 휴식
1500명 완료 == 경과시간: 4954초 == 10초 휴식


In [5]:
# STATIZ와 KBO의 생년월일이 맞지 않는 몇몇을 판별 성공, 이들을 대체
final.loc[(final['선수명'] == '정종수') & (final['생년월일(STATIZ)'] == '1980-06-08'), '생년월일(STATIZ)'] = final.loc[(final['선수명'] == '정종수') & (final['생년월일(KBO)'] == '1980-05-08'), '생년월일(KBO)']
final.loc[(final['선수명'] == '박가람') & (final['생년월일(STATIZ)'] == '1986-11-09'), '생년월일(STATIZ)'] = final.loc[(final['선수명'] == '박가람') & (final['생년월일(KBO)'] == '1986-11-07'), '생년월일(KBO)']
final.loc[(final['선수명'] == '최현종') & (final['생년월일(STATIZ)'] == '1985-08-05'), '생년월일(STATIZ)'] = final.loc[(final['선수명'] == '최현종') & (final['생년월일(KBO)'] == '1985-07-08'), '생년월일(KBO)']
final.loc[(final['선수명'] == '김태석') & (final['생년월일(STATIZ)'] == '1967-02-12'), '생년월일(STATIZ)'] = final.loc[(final['선수명'] == '김태석') & (final['생년월일(KBO)'] == '1967-03-12'), '생년월일(KBO)']
final.loc[(final['선수명'] == '안병원') & (final['생년월일(STATIZ)'] == '1973-10-04'), '생년월일(STATIZ)'] = final.loc[(final['선수명'] == '안병원') & (final['생년월일(KBO)'] == '1973-10-03'), '생년월일(KBO)']
final.loc[(final['선수명'] == '강관식') & (final['생년월일(STATIZ)'] == '1984-04-12'), '생년월일(STATIZ)'] = final.loc[(final['선수명'] == '강관식') & (final['생년월일(KBO)'] == '1981-04-12'), '생년월일(KBO)']
final.loc[(final['선수명'] == '이정용') & (final['생년월일(STATIZ)'] == '1997-03-26'), '생년월일(STATIZ)'] = final.loc[(final['선수명'] == '이정용') & (final['생년월일(KBO)'] == '1996-03-26'), '생년월일(KBO)']
final.loc[(final['선수명'] == '이호중') & (final['생년월일(STATIZ)'] == '1992-10-13'), '생년월일(STATIZ)'] = final.loc[(final['선수명'] == '이호중') & (final['생년월일(KBO)'] == '1992-10-03'), '생년월일(KBO)']
final.loc[(final['선수명'] == '오동욱') & (final['생년월일(STATIZ)'] == '2001-02-01'), '생년월일(STATIZ)'] = final.loc[(final['선수명'] == '오동욱') & (final['생년월일(KBO)'] == '2001-02-04'), '생년월일(KBO)']

In [6]:
# 생년월일 다른 인원이 있는지 체크

"""
statiz: 한동민은 문제가 되지 않음(2020년까지는 한동민, 2021년부터 한유섬)
statiz: 나종덕은 문제가 되지 않음(2020년부터 나균안이었지만, 2020년의 기록이 없음)
"""

final['bool'] = final['생년월일(KBO)'] == final['생년월일(STATIZ)']
final[final['bool'] == False]

,선수명,생년월일(KBO),생년월일(STATIZ),데뷔년도,bool


In [7]:
# 불필요한 열 제거 및 열이름 변경
final = final.drop(['생년월일(STATIZ)', 'bool'], axis=1).rename(columns={'생년월일(KBO)':'생년월일'})
final = final.drop_duplicates(subset=['선수명', '생년월일']).reset_index(drop=True) # 투타겸업했던 인원들은 중복되어 나타났기에 제거
final.loc[final['데뷔년도'] == '없음', '데뷔년도'] = np.nan
final

,선수명,생년월일,데뷔년도
0,김경언,1982-12-07,2001
1,김민철,1982-03-14,2001
2,김상훈,1977-10-27,1996
3,김인철,1971-07-14,1990
4,김종국,1973-09-14,1996
...,...,...,...
1558,김기탁,1998-09-30,2017
1559,오동욱,2001-02-04,NaN
1560,윤대경,1994-04-09,2013
1561,장웅정,1997-04-11,NaN


# STATIZ에서 크롤링한 값으로 데뷔년도 공백 매우기

In [8]:
hitter = pd.read_csv('선수데이터(전처리완료)/타자.csv')
pitcher = pd.read_csv('선수데이터(전처리완료)/투수.csv')
hitter

,ID,선수명,데뷔년도,생년월일,팀명,연도,타율,경기,타석,타수,...,실책(수비),견제사(수비),자살(수비),보살(수비),병살(수비),포일(수비),도루허용(수비),도루실패(수비),수비율(수비),도루저지율(수비)
0,71610,김경언,2001.0,1982-12-07,KIA,2002,0.263,107,323,274,...,1.0,0.0,160.0,8.0,4.0,0.0,0.0,0.0,0.992142,0.0
1,71609,김민철,2001.0,1982-03-14,KIA,2002,0.261,58,53,46,...,1.0,0.0,12.0,17.0,4.0,0.0,0.0,0.0,0.937317,0.0
2,70612,김상훈,1996.0,1977-10-27,KIA,2002,0.221,113,391,357,...,6.0,1.0,664.0,54.0,8.0,8.0,48.0,44.0,0.992000,47.8
3,90811,김인철,NaN,1971-07-14,KIA,2002,0.235,37,53,51,...,2.0,0.0,37.0,1.0,0.0,0.0,0.0,0.0,0.950368,0.0
4,96616,김종국,1996.0,1973-09-14,KIA,2002,0.287,133,595,527,...,9.0,0.0,286.0,421.0,83.0,0.0,0.0,0.0,0.987000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4234,50707,최인호,2020.0,2000-01-30,한화,2020,0.236,47,135,127,...,0.0,0.0,67.0,4.0,0.0,0.0,0.0,0.0,0.996250,0.0
4235,78288,최재훈,2008.0,1989-08-27,한화,2020,0.301,126,389,339,...,5.0,1.0,727.0,48.0,12.0,4.0,79.0,27.0,0.994000,25.5
4236,74729,최진행,2004.0,1985-08-17,한화,2020,0.234,84,240,214,...,1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.970000,0.0
4237,62700,하주석,2012.0,1994-02-25,한화,2020,0.286,72,281,262,...,9.0,0.0,89.0,148.0,37.0,0.0,0.0,0.0,0.963000,0.0


In [9]:
# 타자의 데뷔년도 결측치 채우기
for name, year in zip(hitter.loc[hitter['데뷔년도'].isnull(), '선수명'], hitter.loc[hitter['데뷔년도'].isnull(), '생년월일']):
    try:
        hitter.loc[(hitter['선수명'] == name) & (hitter['생년월일'] == year), '데뷔년도'] = int(final.loc[(final['선수명'] == name) & (final['생년월일'] == year), '데뷔년도'])
    except:
        print(f'== {name}({year}) 오류 ==')

== 이일의(1971-11-30) 오류 ==
== 김대원(1978-05-18) 오류 ==
== 조정권(1976-02-25) 오류 ==
== 김필중(1975-05-16) 오류 ==
== 최익성(1972-02-11) 오류 ==
== 이일의(1971-11-30) 오류 ==
== 김영화(1965-11-20) 오류 ==
== 고지행(1978-05-03) 오류 ==
== 김무성(1978-03-02) 오류 ==
== 조정권(1976-02-25) 오류 ==
== 최익성(1972-02-11) 오류 ==
== 고지행(1978-05-03) 오류 ==
== 최익성(1972-02-11) 오류 ==
== 이송영(1979-01-17) 오류 ==
== 최익성(1972-02-11) 오류 ==
== 권민성(1981-05-31) 오류 ==
== 고지행(1978-05-03) 오류 ==
== 유덕형(1984-05-26) 오류 ==
== 박가람(1986-11-07) 오류 ==
== 송수(1984-03-04) 오류 ==
== 이강서(1984-01-10) 오류 ==
== 김혜겸(1985-01-05) 오류 ==
== 유덕형(1984-05-26) 오류 ==
== 강병수(1984-04-16) 오류 ==
== 이성호(1985-03-09) 오류 ==
== 안동현(1986-12-24) 오류 ==
== 임경훈(1989-08-30) 오류 ==


In [10]:
# 투수의 데뷔년도 결측치 채우기
for name, year in zip(pitcher.loc[pitcher['데뷔년도'].isnull(), '선수명'], pitcher.loc[pitcher['데뷔년도'].isnull(), '생년월일']):
    try:
        pitcher.loc[(pitcher['선수명'] == name) & (pitcher['생년월일'] == year), '데뷔년도'] = int(final.loc[(final['선수명'] == name) & (final['생년월일'] == year), '데뷔년도'])
    except:
        print(f'== {name}({year}) 오류 ==')

== 이용호(1970-05-06) 오류 ==
== 박지철(1975-04-13) 오류 ==
== 문남열(1978-10-17) 오류 ==
== 김병준(1977-04-01) 오류 ==
== 한용덕(1965-06-02) 오류 ==
== 권준헌(1971-12-01) 오류 ==
== 최환인(1979-06-15) 오류 ==
== 김진유(1979-07-01) 오류 ==
== 박지철(1975-04-13) 오류 ==
== 이석만(1980-05-08) 오류 ==
== 문남열(1978-10-17) 오류 ==
== 한용덕(1965-06-02) 오류 ==
== 권준헌(1971-12-01) 오류 ==
== 김진유(1979-07-01) 오류 ==
== 김종민(1981-06-30) 오류 ==
== 박지철(1975-04-13) 오류 ==
== 권준헌(1971-12-01) 오류 ==
== 한용덕(1965-06-02) 오류 ==
== 김종민(1981-06-30) 오류 ==
== 최혁길(1981-03-25) 오류 ==
== 박지철(1975-04-13) 오류 ==
== 이석만(1980-05-08) 오류 ==
== 서민욱(1982-01-09) 오류 ==
== 조성민(1973-04-05) 오류 ==
== 이상우(1981-08-13) 오류 ==
== 김종민(1981-06-30) 오류 ==
== 김태환(1985-03-30) 오류 ==
== 박지철(1975-04-13) 오류 ==
== 권준헌(1971-12-01) 오류 ==
== 서민욱(1982-01-09) 오류 ==
== 조성민(1973-04-05) 오류 ==
== 박지철(1975-04-13) 오류 ==
== 권준헌(1971-12-01) 오류 ==
== 서민욱(1982-01-09) 오류 ==
== 조성민(1973-04-05) 오류 ==
== 권준헌(1971-12-01) 오류 ==
== 박지철(1975-04-13) 오류 ==
== 박찬호(1973-06-30) 오류 ==


In [11]:
# 못채운 고유선수들 파악
print(f"투수: {pitcher[pitcher['데뷔년도'].isnull()].drop_duplicates(subset=['선수명','생년월일']).shape[0]}명")
print(f"타자: {hitter[hitter['데뷔년도'].isnull()].drop_duplicates(subset=['선수명','생년월일']).shape[0]}명")

투수: 16명
타자: 19명


In [12]:
pitcher[pitcher['데뷔년도'].isnull()].drop_duplicates(subset=['선수명','생년월일']).reset_index(drop=True)
hitter[hitter['데뷔년도'].isnull()].drop_duplicates(subset=['선수명','생년월일']).reset_index(drop=True)

,ID,선수명,데뷔년도,생년월일,팀명,연도,타율,경기,타석,타수,...,실책(수비),견제사(수비),자살(수비),보살(수비),병살(수비),포일(수비),도루허용(수비),도루실패(수비),수비율(수비),도루저지율(수비)
0,72195,이일의,NaN,1971-11-30,LG,2002,0.246,72,137,122,...,0.0,0.0,19.0,1.0,0.0,0.0,0.0,0.0,1.000000,0.0
1,71761,김대원,NaN,1978-05-18,한화,2002,0.000,15,2,2,...,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,1.000000,0.0
2,71463,조정권,NaN,1976-02-25,한화,2002,0.167,15,12,12,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.400000,0.0
3,99302,김필중,NaN,1975-05-16,현대,2002,0.000,3,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,94426,최익성,NaN,1972-02-11,현대,2002,0.280,45,95,75,...,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.970297,0.0
5,73540,김영화,NaN,1965-11-20,롯데,2003,0.111,7,9,9,...,0.0,0.0,8.0,0.0,0.0,1.0,0.0,0.0,1.000000,0.0
6,73732,고지행,NaN,1978-05-03,삼성,2003,0.281,92,289,253,...,8.0,0.0,128.0,160.0,30.0,0.0,0.0,0.0,0.975176,0.0
7,71763,김무성,NaN,1978-03-02,한화,2003,0.000,2,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
8,71744,이송영,NaN,1979-01-17,한화,2004,1.000,1,1,1,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.000000,0.0
9,71650,권민성,NaN,1981-05-31,롯데,2005,0.231,10,14,13,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0


In [13]:
# pitcher[pitcher['데뷔년도'].isnull()].drop_duplicates(subset=['선수명','생년월일']).reset_index(drop=True).to_csv('투수_데뷔년도결측_수기조사필요.csv', index=False, encoding='utf-8-sig')
# hitter[hitter['데뷔년도'].isnull()].drop_duplicates(subset=['선수명','생년월일']).reset_index(drop=True).to_csv('타자_데뷔년도결측_수기조사필요.csv', index=False, encoding='utf-8-sig')

# 이 외 결측치는 직접 인터넷에서 찾아 수기대입

In [14]:
hitter_nan = pd.read_csv('데뷔년도결측치수기조사데이터/타자_데뷔년도결측_수기조사필요.csv', encoding='cp949')[['선수명','데뷔년도','생년월일']]
pitcher_nan = pd.read_csv('데뷔년도결측치수기조사데이터/투수_데뷔년도결측_수기조사필요.csv', encoding='cp949')[['선수명','데뷔년도','생년월일']]
pitcher_nan.head()

,선수명,데뷔년도,생년월일
0,박지철,1994,1975-04-13
1,문남열,1999,1978-10-17
2,김병준,1999,1977-04-01
3,권준헌,1990,1971-12-01
4,최환인,2002,1979-06-15


In [15]:
# 타자 대체
for name, year, birthday in zip(hitter_nan['선수명'], hitter_nan['데뷔년도'], hitter_nan['생년월일']):
    hitter.loc[(hitter['선수명'] == name) & (hitter['생년월일'] == birthday), '데뷔년도'] = int(year)

In [16]:
# 투수 대체
for name, year, birthday in zip(pitcher_nan['선수명'], pitcher_nan['데뷔년도'], pitcher_nan['생년월일']):
    pitcher.loc[(pitcher['선수명'] == name) & (pitcher['생년월일'] == birthday), '데뷔년도'] = int(year)

In [17]:
hitter.to_csv('선수데이터(전처리완료)/타자.csv', index=False, encoding='utf-8-sig')
pitcher.to_csv('선수데이터(전처리완료)/투수.csv', index=False, encoding='utf-8-sig')